# Systematics Notebook
From results of the phi fitting.

In [1]:
import glob 
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import pickle
import sys
import time 

%matplotlib inline

plt.rc('font', family='serif')
plt.rc('font', size=18)

### Load Configurations
There are several files with different results for phi-distributions.  

In [2]:
database_files = glob.glob('database/fit/*.csv')
print('Found %d files in the database.' % len(database_files))

Found 24 files in the database.


In [3]:
def load_database_files(file_list):
    
    dataframe_store = {}
    for f in file_list:
        dataframe_store[f] = pd.read_csv(f)
        
    return dataframe_store

In [4]:
dataframe_store = load_database_files(database_files)

Finally, fit the nominal phi distributions.

In [5]:
nominal = pd.read_csv('database/fit/sys.csv')

### Systematic Uncertainties 

In [6]:
def database_filename_parser(file_name):
    file_name = file_name.split('variation_')[-1].strip('.csv')
    
    tokens = file_name.split('_')
    n_tokens = len(tokens)

    parameter_name = '_'.join(tokens[0:n_tokens-1])
    index = int(tokens[-1])
    
    return index, parameter_name

In [7]:
def build_parameter_variation_dict(path_to_db = 'database/fit/'):

    parameters = {}

    database_files = glob.glob(path_to_db + 'variation*.csv')
    
    for database_file in database_files:
        index, parameter = database_filename_parser(database_file)
        
        if parameter in parameters.keys():
            parameters[parameter][index] = pd.read_csv(database_file)
        else:
            parameters[parameter] = {}
            parameters[parameter][index] = pd.read_csv(database_file)
            
    return parameters

In [8]:
def load_systematic_sources_list(file_name):
    systematic_sources = pickle.load(open(file_name, 'rb'))
    
    reverse_dict = {}

    for key, value in systematic_sources.iteritems():
        reverse_dict[value] = key
    
    return reverse_dict

In [9]:
parameter_variation = build_parameter_variation_dict()

In [10]:
for key, value in parameter_variation.iteritems():
    print(key, value.keys())

('dist_dcr3', [1, -1])
('dist_dcr1', [1, -1])
('dist_vz', [1, -1])
('dist_ecw', [1, -1])
('dist_ec_edep', [1, -1])
('dist_ecsf', [1, -1])
('p_mes', [0, -1])
('alpha', [1, -1])
('dist_ecu', [1, -1])
('dist_cc', [0, -1])
('dist_ecv', [1, -1])


In [11]:
systematic_sources = load_systematic_sources_list('systematic_sources.pkl')

In [14]:
def add_systematics(nominal_fit, parameter_variation, systematic_sources):

    nominal_fit_sys = nominal_fit.copy(deep=True)
    nominal_fit_sys['sys_total_0'] = np.zeros(len(nominal_fit_sys))
    nominal_fit_sys['sys_total_1'] = np.zeros(len(nominal_fit_sys))
    nominal_fit_sys['sys_total_2'] = np.zeros(len(nominal_fit_sys))
    
    for key in parameter_variation.keys():

        min_index = parameter_variation[key].keys()[0]    
        max_index = parameter_variation[key].keys()[-1]

        merged_data = pd.merge(parameter_variation[key][min_index], 
                 parameter_variation[key][max_index],
                 on = ['axis', 'axis_bin'])

        merged_data[systematic_sources[key] + '_par0'] = np.abs(merged_data.par_0_y - merged_data.par_0_x)
        merged_data[systematic_sources[key] + '_par1'] = np.abs(merged_data.par_1_y - merged_data.par_1_x)
        merged_data[systematic_sources[key] + '_par2'] = np.abs(merged_data.par_2_y - merged_data.par_2_x)

        merge_cols = ['axis', 'axis_bin', systematic_sources[key] + '_par0', 
                     systematic_sources[key] + '_par1', systematic_sources[key] + '_par2']

        nominal_fit_sys = pd.merge(nominal_fit_sys, merged_data[merge_cols], 
                                   on = ['axis', 'axis_bin'])
    
        nominal_fit_sys.sys_total_0 += nominal_fit_sys[systematic_sources[key] + '_par0']**2
        nominal_fit_sys.sys_total_1 += nominal_fit_sys[systematic_sources[key] + '_par1']**2
        nominal_fit_sys.sys_total_2 += nominal_fit_sys[systematic_sources[key] + '_par2']**2


    nominal_fit_sys.sys_total_0 = np.sqrt(nominal_fit_sys.sys_total_0)
    nominal_fit_sys.sys_total_1 = np.sqrt(nominal_fit_sys.sys_total_1)
    nominal_fit_sys.sys_total_2 = np.sqrt(nominal_fit_sys.sys_total_2)

    
    return nominal_fit_sys

In [15]:
nominal_fit_sys = add_systematics(nominal, parameter_variation, systematic_sources)

In [16]:
nominal_fit_sys.to_csv('results/fit/sys.csv', index=False)

In [17]:
nominal_fit_sys.head(24)

axis  axis_bin  axis_max  axis_min     err_0     err_1     err_2     par_0  \
0    pt         0  0.202108  0.000284  0.003848  0.201847  0.201488  0.024732   
1    pt         1  0.274544  0.202108  0.004326  0.161944  0.150773  0.026399   
2    pt         2  0.328453  0.274544  0.004196  0.185239  0.182566  0.030660   
3    pt         3  0.376169  0.328453  0.004066  0.168019  0.144424  0.034853   
4    pt         4  0.422123  0.376169  0.004451  0.154862  0.171526  0.036418   
5    pt         5  0.469541  0.422123  0.004105  0.149596  0.139497  0.033018   
6    pt         6  0.521940  0.469541  0.004933  0.192376  0.157255  0.036059   
7    pt         7  0.585257  0.521940  0.003973  0.156984  0.152007  0.035456   
8    pt         8  0.674587  0.585257  0.004647  0.156082  0.188118  0.032423   
9    pt         9  1.193084  0.674587  0.003853  0.197304  0.190875  0.020430   
10   q2         0  1.173185  1.000001  0.003639  0.244998  0.269459  0.027681   
11   q2         1  1.299205  1.173185  0.003489  0.148336  0.135190  0.033268   
12   q2         2  1.416112  1.299205  0.004308  0.167681  0.177845  0.026427   
13   q2         3  1.539892  1.416112  0.004162  0.183290  0.177284  0.029325   
14   q2         4  1.679987  1.539892  0.004308  0.151145  0.160940  0.032128   
15   q2         5  1.845621  1.679987  0.004577  0.164385  0.167332  0.031012   
16   q2         6  2.051025  1.845621  0.004436  0.150760  0.169013  0.034882   
17   q2         7  2.321366  2.051025  0.004266  0.177279  0.187192  0.027995   
18   q2         8  2.734286  2.321366  0.004271  0.169378  0.150847  0.036262   
19   q2         9  4.736822  2.734286  0.004768  0.174330  0.156161  0.029296   
20    x         0  0.143282  0.106729  0.003474  0.288162  0.380403  0.023518   
21    x         1  0.166495  0.143282  0.003940  0.217645  0.225949  0.023211   
22    x         2  0.188544  0.166495  0.003931  0.145143  0.133163  0.036944   
23    x         3  0.210731  0.188544  0.003594  0.144718  0.159225  0.033895   

       par_1     par_2     ...       sys_4_par2  sys_3_par0  sys_3_par1  \
0   0.018225 -0.009308     ...         0.009210    0.001102    0.072911   
1  -0.499265  0.269604     ...         0.027505    0.003593    0.126185   
2   0.135400 -0.147474     ...         0.055675    0.000413    0.010151   
3  -0.131565  0.006722     ...         0.007738    0.001698    0.017638   
4   0.045705 -0.059875     ...         0.076107    0.002113    0.052837   
5  -0.369652  0.061894     ...         0.008088    0.001405    0.154185   
6   0.243873 -0.089681     ...         0.039093    0.002695    0.070319   
7  -0.211701 -0.081571     ...         0.021728    0.001696    0.138483   
8  -0.306766  0.056407     ...         0.022820    0.005322    0.048114   
9  -0.456216  0.190335     ...         0.054580    0.000189    0.058277   
10 -0.142137 -0.154216     ...         0.066569    0.004052    0.414280   
11 -0.082297 -0.351990     ...         0.047165    0.004817    0.133917   
12 -0.403022  0.152441     ...         0.039808    0.002491    0.028186   
13 -0.205406 -0.001454     ...         0.010768    0.001991    0.008706   
14 -0.356609  0.083419     ...         0.024866    0.000019    0.021975   
15 -0.272939  0.141079     ...         0.017891    0.000020    0.001363   
16 -0.032646  0.051051     ...         0.024289    0.000061    0.000037   
17 -0.231326  0.070724     ...         0.049302    0.000036    0.005023   
18 -0.027417 -0.011032     ...         0.047923    0.000356    0.013614   
19  0.143708  0.256770     ...         0.049131    0.001027    0.043930   
20 -0.547084 -0.541683     ...         0.160268    0.000262    0.343251   
21 -0.011143 -0.122404     ...         0.031214    0.004250    0.001790   
22 -0.238006 -0.025445     ...         0.044430    0.002936    0.034999   
23 -0.251026 -0.380708     ...         0.032298    0.002026    0.026455   

    sys_3_par2  sys_5_par0  sys_5_par1  sys_5_par2  sys_11_par0  sys_11_par1  \
0     0.093322    0.0